In [1]:
from sqlalchemy import create_engine, URL, text
from dotenv import load_dotenv
import pandas as pd
import os

In [2]:
load_dotenv()

pg_user = os.environ["POSTGRES_USER"]
pg_db = os.environ["POSTGRES_DB"]

url = URL.create(
  "postgresql+psycopg2",
  username=pg_user,
  database=pg_db
)

engine = create_engine(url)

#### Get dimension tables

In [3]:
def delete_dim_tables(list_dims: list[str], 
                      keep: list[str] | set[str]) -> None:
  set_keep = set(keep)

  with engine.connect() as con:
    for dim_tbl in list_dims:
      if dim_tbl in set_keep:
        continue

      query = text(f"DROP TABLE {dim_tbl}")
      try:
        con.execute(query)
      except:
        print(f"Couldn't drop table {dim_tbl}")

    con.commit()

In [4]:
def get_dim_tables(base: str) -> list[str]:
  query = text(f"""SELECT table_name 
                  FROM information_schema.tables 
                  WHERE table_name LIKE '{base}_%'""")
  
  return pd.read_sql_query(query, engine)["table_name"]

In [5]:
def get_list_keys(tbl: str) -> list[str]:
  query = text(f"""SELECT *
                  FROM {tbl}
                  LIMIT 0 """)

  return list(pd.read_sql_query(query, engine)
                .keys())

In [6]:
list_dim_tables = get_dim_tables("bop")
list_keys = get_list_keys("balance_of_pay")

In [7]:
keep_list = set(["bop_measure", "bop_ref_area", "bop_currency"])
delete_dim_tables(list_dim_tables, keep_list)

In [8]:
list_dim_tables = get_dim_tables("int_")
list_keys = get_list_keys("interest_rate")
list_keys

['entity', 'measure', 'date', 'value']

In [9]:
list_keep = set(["int_rates_measure", "int_rates_ref_area", "interest_rate"])
delete_dim_tables(list_dim_tables, keep = list_keep)

In [10]:
list_dim_tables = get_dim_tables("ex_rates_%")
list_keys = get_list_keys("ex_rates")
list_keys

['currency', 'time_period', 'rate']

In [11]:
delete_dim_tables(list_dim_tables, keep = set(["ex_rates_currency"]))